In [1]:
from pathlib import Path
from power_perceiver.load_prepared_batches.data_loader import PV
import numpy as np

In [2]:
DATA_PATH = Path("~/dev/ocf/power_perceiver/data_for_testing/").expanduser()

In [3]:
pv = PV(data_path=DATA_PATH)

In [4]:
pv_batch = pv[0]
pv_batch

<xarray.Dataset>
Dimensions:               (example: 32, pv_system: 128, time: 31)
Coordinates:
    pv_system_id          (example, pv_system) float64 2.766e+03 ... nan
    pv_system_row_number  (example, pv_system) int64 7 52 59 74 87 ... 0 0 0 0 0
    time_utc              (example, time) datetime64[ns] 2020-09-12T09:05:00 ...
    x_osgb                (example, pv_system) float32 2.208e+05 ... nan
    y_osgb                (example, pv_system) float32 6.608e+05 ... nan
Dimensions without coordinates: example, pv_system, time
Data variables:
    capacity_wp           (example, pv_system) float32 2.176e+03 3e+03 ... nan
    power_w               (example, time, pv_system) float32 246.5 811.9 ... nan
    power_normalised      (example, time, pv_system) float32 0.1133 ... nan
    pv_mask               (example, pv_system) bool True True ... False False

In [17]:
pv_batch["time_utc"].where(pv_batch.pv_mask)

<xarray.DataArray 'time_utc' (example: 32, time: 31, pv_system: 128)>
array([[['2020-09-12T09:05:00.000000000',
         '2020-09-12T09:05:00.000000000',
         '2020-09-12T09:05:00.000000000', ...,
                                   'NaT',
                                   'NaT',
                                   'NaT'],
        ['2020-09-12T09:10:00.000000000',
         '2020-09-12T09:10:00.000000000',
         '2020-09-12T09:10:00.000000000', ...,
                                   'NaT',
                                   'NaT',
                                   'NaT'],
        ['2020-09-12T09:15:00.000000000',
         '2020-09-12T09:15:00.000000000',
         '2020-09-12T09:15:00.000000000', ...,
                                   'NaT',
                                   'NaT',
                                   'NaT'],
        ...,
        ['2020-09-12T11:25:00.000000000',
...
                                   'NaT'],
        ...,
        ['2020-06-17T15:35:00.000000000',
         '2020-06-17T15:35:00.000000000',
         '2020-06-17T15:35:00.000000000', ...,
                                   'NaT',
                                   'NaT',
                                   'NaT'],
        ['2020-06-17T15:40:00.000000000',
         '2020-06-17T15:40:00.000000000',
         '2020-06-17T15:40:00.000000000', ...,
                                   'NaT',
                                   'NaT',
                                   'NaT'],
        ['2020-06-17T15:45:00.000000000',
         '2020-06-17T15:45:00.000000000',
         '2020-06-17T15:45:00.000000000', ...,
                                   'NaT',
                                   'NaT',
                                   'NaT']]], dtype='datetime64[ns]')
Coordinates:
    time_utc              (example, time) datetime64[ns] 2020-09-12T09:05:00 ...
    pv_system_id          (example, pv_system) float64 2.766e+03 ... nan
    pv_system_row_number  (example, pv_system) int64 7 52 59 74 87 ... 0 0 0 0 0
    x_osgb                (example, pv_system) float32 2.208e+05 ... 0.0
    y_osgb                (example, pv_system) float32 6.608e+05 ... 0.0
Dimensions without coordinates: example, time, pv_system

In [61]:
rng = np.random.default_rng()
requested_num_pv_systems = 8
num_examples = len(pv_batch.example)

selection = np.zeros(shape=(num_examples, requested_num_pv_systems), dtype=np.int32)
for example_i in range(num_examples):
    pv_mask_for_example = pv_batch.pv_mask.isel(example=example_i).values
    all_indicies = np.nonzero(pv_mask_for_example)[0]
    # Only allow a PV system to be chosen multiple times for this example if there are
    # less available PV systems than requested PV systems.
    replace = len(all_indicies) < requested_num_pv_systems
    chosen_indicies = rng.choice(all_indicies, requested_num_pv_systems, replace=replace)
    selection[example_i] = chosen_indicies

In [65]:
import xarray as xr
selection = xr.DataArray(selection, dims=("example", "pv_system"))
selection

<xarray.DataArray (example: 32, pv_system: 8)>
array([[  0,  22,  54,  30,   2,  32,  46,   5],
       [  0,  16,   6,   1,   5,  13,  12,  10],
       [ 32,  35,  52,  66,  61,  18,  82,  88],
       [ 69,   4,  53,  13,  28,  57,  33,  92],
       [  9,   1,   5,   8,   2,   0,   3,   7],
       [  2,  57,  17,  94,  20,  66,  46,  69],
       [ 35,  52,  62,  15,  48,  32,  51,   1],
       [ 17,  89,  14,  70,  41,  68,  16,  23],
       [  7,   0,   9,  10,   3,   2,   5,   4],
       [ 29,  20,  14,  33,   5,  10,  31,  39],
       [ 31,   7,   6,  27,  35,  12,  16,  11],
       [  5,  12,  25,   7,  28,  20,  17,   1],
       [ 10,  21,  37,  12,   8,  19,  39,  11],
       [ 74,  61,  50,  11,  28,  76,  12,   4],
       [  5,   0,   4,   3,   0,   3,   1,   4],
       [ 24,  49,  78,  92, 100,  27,  98,  36],
       [ 17,  62,  36,  72,  92,  63,  90,  68],
       [ 58,  64,  35,  82,   3,  78,  79,  56],
       [ 25,  33,  17,   9,  36,   5,   0,   3],
       [ 64,   7,  59,   6,  54,  71,  51,  41],
       [  0,  79,  10,  34,  92,  52,  21,   6],
       [  1,   8,   6,  12,   4,  10,   2,   0],
       [ 14,  32,   6,  27,  12,   5,  17,  22],
       [ 25,  61,  75,  11,  48,   0,  39,  34],
       [  6,  12,   4,   9,   7,  16,  27,  18],
       [ 23,  36,  34,  11,  45,  17,  21,  14],
       [ 22,   7,  21,   5,   0,   9,  13,  20],
       [  5,  74,  39,  14,   3,  60,  94,  43],
       [ 76,  12,  98,  97,  35,  44,  43,  65],
       [ 53,  17,  34,  72,  22,  41,  46,   2],
       [  3,  20,  32,  46,   2,  41,  33,  16],
       [  2,  21,  18,  29,  52,  44,  55,  39]], dtype=int32)
Dimensions without coordinates: example, pv_system

In [70]:
new_pv_batch = pv_batch.isel(pv_system=selection)
new_pv_batch

<xarray.Dataset>
Dimensions:               (example: 32, pv_system: 8, time: 31)
Coordinates:
    pv_system_id          (example, pv_system) float64 2.766e+03 ... 9.867e+03
    pv_system_row_number  (example, pv_system) int64 7 373 725 ... 728 882 535
    time_utc              (example, time) datetime64[ns] 2020-09-12T09:05:00 ...
    x_osgb                (example, pv_system) float32 2.208e+05 ... 4.057e+05
    y_osgb                (example, pv_system) float32 6.608e+05 ... 4.253e+05
Dimensions without coordinates: example, pv_system, time
Data variables:
    capacity_wp           (example, pv_system) float32 2.176e+03 ... 2.225e+03
    power_w               (example, time, pv_system) float32 246.5 0.0 ... 998.5
    power_normalised      (example, time, pv_system) float32 0.1133 ... 0.4487
    pv_mask               (example, pv_system) bool True True True ... True True

In [77]:
new_pv_batch.where(new_pv_batch.pv_mask)

<xarray.Dataset>
Dimensions:               (example: 32, pv_system: 8, time: 31)
Coordinates:
    pv_system_id          (example, pv_system) float64 2.766e+03 ... 9.867e+03
    pv_system_row_number  (example, pv_system) int64 7 373 725 ... 728 882 535
    time_utc              (example, time) datetime64[ns] 2020-09-12T09:05:00 ...
    x_osgb                (example, pv_system) float32 2.208e+05 ... 4.057e+05
    y_osgb                (example, pv_system) float32 6.608e+05 ... 4.253e+05
Dimensions without coordinates: example, pv_system, time
Data variables:
    capacity_wp           (example, pv_system) float32 2.176e+03 ... 2.225e+03
    power_w               (example, time, pv_system) float32 246.5 0.0 ... 998.5
    power_normalised      (example, time, pv_system) float32 0.1133 ... 0.4487
    pv_mask               (example, pv_system) float64 1.0 1.0 1.0 ... 1.0 1.0

In [21]:
indicies = pv_batch.pv_system.where(pv_batch.pv_mask).T
indicies

<xarray.DataArray 'pv_system' (example: 32, pv_system: 128)>
array([[ 0.,  1.,  2., ..., nan, nan, nan],
       [ 0.,  1.,  2., ..., nan, nan, nan],
       [ 0.,  1.,  2., ..., nan, nan, nan],
       ...,
       [ 0.,  1.,  2., ..., nan, nan, nan],
       [ 0.,  1.,  2., ..., nan, nan, nan],
       [ 0.,  1.,  2., ..., nan, nan, nan]])
Coordinates:
    pv_system_id          (pv_system, example) float64 2.766e+03 ... nan
    pv_system_row_number  (pv_system, example) int64 7 21 3 3 4 2 ... 0 0 0 0 0
    x_osgb                (pv_system, example) float32 2.208e+05 ... nan
    y_osgb                (pv_system, example) float32 6.608e+05 ... nan
Dimensions without coordinates: example, pv_system

In [45]:
rng = np.random.default_rng()
rng.choice(indicies.values, size=8, axis=1)

array([[nan, 26., 24., nan, 40., 17., nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan],
       [57., 26., 24., 87., 40., 17., 91., 75.],
       [57., 26., 24., 87., 40., 17., 91., 75.],
       [nan, nan, nan, nan, nan, nan, nan, nan],
       [57., 26., 24., 87., 40., 17., 91., 75.],
       [57., 26., 24., nan, 40., 17., nan, nan],
       [57., 26., 24., 87., 40., 17., 91., 75.],
       [nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, 26., 24., nan, 40., 17., nan, nan],
       [nan, 26., 24., nan, 40., 17., nan, nan],
       [nan, 26., 24., nan, nan, 17., nan, nan],
       [nan, 26., 24., nan, nan, 17., nan, nan],
       [57., 26., 24., nan, 40., 17., nan, 75.],
       [nan, nan, nan, nan, nan, nan, nan, nan],
       [57., 26., 24., 87., 40., 17., 91., 75.],
       [57., 26., 24., 87., 40., 17., 91., 75.],
       [57., 26., 24., nan, 40., 17., nan, 75.],
       [nan, 26., 24., nan, 40., 17., nan, nan],
       [57., 26., 24., nan, 40., 17., nan, nan],
       [57., 26., 24